In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
from nltk import ngrams
from collections import defaultdict
import nltk
import collections
import torch

In [ ]:
# Read data 
X_train = open('x_train.txt',encoding="utf8").read().split('\n')
Y_train = open('y_train.txt',encoding="utf8").read().split('\n')
labels = pd.read_csv('labels.csv',delimiter = ';')
Y_train =  Y_train[:-1]
X_train =  X_train[:-1]

In [ ]:
# Remove unnecessary characters from data
extras = '!"$%&/{}[]()=?\\`´*+~#-_.:,;<>|1234567890°-\'' # Characters to remove from data
rx = '[' + re.escape(''.join(extras)) + ']'
x_train =[] 
for example in X_train:
    x_train.append(re.sub(' +', ' ',re.sub(rx, '', example)))

In [ ]:
# Convert language labels to language Name => 'en' -> 'English'
lab_dict = { labels.loc[i]['Label']:labels.loc[i]['English'] for i in range(0,len(labels))}
y_train = [lab_dict[item] if item != 'nan' else 'Min Nan Chinese' for item in Y_train ]

In [ ]:
# Collecting examples belonging to same language
# lang_2_ex = { 'English' : [ 'Welcome to ..', 'PHP sucks ...', ...] , 'Spanish' : ['dads....', 'dasdsa...',....] , ....}

lang_2_ex = defaultdict(list)
for i in range(len(x_train)):
  lang_2_ex[y_train[i]].append(x_train[i])

In [ ]:
# Creating ngrams for each language 
n=3 # trigram
gram_per_lang = defaultdict(list)
uniq_lang = set(y_train)
for lang in uniq_lang:
  for sent in lang_2_ex[lang]:
    gram_per_lang[lang]+=[sent[i:i+n] for i in range(len(sent)-n+1)]

In [ ]:
# Calculating frequency distribution for ngrams per language

freq_per_lang = defaultdict(list)
for lang in uniq_lang:
  freq_per_lang[lang]=dict(zip(list(collections.Counter(gram_per_lang[lang]).keys()),list(collections.Counter(gram_per_lang[lang]).values()))) 

# https://stackoverflow.com/questions/12282232/how-do-i-count-unique-values-inside-a-list
# https://stackoverflow.com/questions/209840/how-do-i-convert-two-lists-into-a-dictionary

In [ ]:
# Sorted Frequency Distribution 

sorted_freq_per_lang = defaultdict(list)
for lang in uniq_lang:
  sorted_freq_per_lang[lang] = {word : value for word,value in sorted(freq_per_lang[lang].items(),key=lambda item : item[1],reverse=True)}

In [ ]:
sorted_freq_per_lang['English']

{' th': 2838,
 'he ': 2765,
 'the': 2569,
 'ed ': 1660,
 ' in': 1371,
 ' an': 1326,
 'nd ': 1287,
 'and': 1277,
 ' of': 1240,
 'of ': 1182,
 'in ': 1102,
 'er ': 1080,
 'on ': 1031,
 'ing': 1015,
 'ng ': 983,
 'as ': 923,
 ' to': 913,
 'es ': 868,
 'to ': 845,
 'ion': 838,
 's a': 836,
 ' a ': 787,
 'ent': 704,
 'is ': 681,
 'tio': 635,
 'or ': 634,
 'n t': 597,
 ' co': 595,
 'al ': 593,
 'd t': 593,
 ' re': 576,
 're ': 560,
 'an ': 551,
 ' fo': 538,
 'e a': 534,
 'for': 527,
 'ter': 522,
 ' wa': 520,
 'e t': 509,
 'at ': 499,
 'ate': 495,
 'ati': 482,
 'st ': 466,
 'e s': 458,
 'was': 457,
 's t': 442,
 'th ': 441,
 'nt ': 435,
 'd a': 431,
 'ly ': 430,
 'ted': 429,
 'The': 427,
 'n a': 425,
 'ts ': 409,
 'f t': 402,
 's o': 395,
 ' Th': 388,
 'e o': 386,
 ' on': 385,
 ' pr': 384,
 ' be': 375,
 ' as': 374,
 ' de': 372,
 'ers': 367,
 't t': 365,
 ' wh': 365,
 'en ': 361,
 'her': 352,
 ' wi': 341,
 'ere': 332,
 'rs ': 330,
 'res': 327,
 'e i': 327,
 ' st': 326,
 'ver': 324,
 'e w': 320